## *Import Libraries*

In [1]:
import cv2
import numpy as np
import json
import matplotlib.pyplot as plt
from keras.models import load_model, model_from_json
from keras.preprocessing import image as img
import os
import sys
import time

## *Cascading*

In [2]:
face_cascade = cv2.CascadeClassifier("haarcascades/haarcascade_frontalface_default.xml")
eye_cascade = cv2.CascadeClassifier("haarcascades/haarcascade_eye.xml")
side_profile = cv2.CascadeClassifier("haarcascades/haarcascade_profileface.xml")
glass_cascade = cv2.CascadeClassifier("haarcascades/haarcascade_eye_tree_eyeglasses.xml")

## *Brightness and Contrast (Alpha Beta) Correction *

In [3]:
def brightness_correction(image):
    filtered_image = np.zeros(image.shape, image.dtype)
    alpha = 1
    beta = 50
    for y in range(image.shape[0]):
        for x in range(image.shape[1]):
            for c in range(image.shape[2]):
                filtered_image[y, x, c] = np.clip(alpha * image[y, x, c] + beta, 0, 255)
    return filtered_image

## *Gamma Correction*

In [4]:
def linear_stretching(input, lower_stretch_from, upper_stretch_from):
    lower_stretch_to = 0  
    upper_stretch_to = 255
    output = (input - lower_stretch_from) * ((upper_stretch_to - lower_stretch_to) / (upper_stretch_from - lower_stretch_from)) + lower_stretch_to
    return output

def gamma_correction(image):
    gamma_image = image.copy()
    max_value = np.max(gamma_image)
    min_value = np.min(gamma_image)
    for y in range(len(gamma_image)):
        for x in range(len(gamma_image[y])):
            gamma_image[y][x] = linear_stretching(gamma_image[y][x], min_value, max_value)
    return gamma_image


## *Fixing/Normalizing the images*

In [5]:
def normalize_image(image):
    image = image.copy()
    output = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX)
    dst = cv2.equalizeHist(output)
    image = dst.copy()
    return image


## *Function to detect Face and eyes*

In [6]:
# Function to detect face
def detect_face(image):
    try:
        face_image = image.copy()
        face_rect = face_cascade.detectMultiScale(face_image, scaleFactor=1.2, minNeighbors=5)
        face_cords =[]
        face_detection = 0
        for (x, y, w, h) in face_rect:
            cv2.rectangle(face_image, (x, y), (x + w, y + h), (255, 255, 255), 10)
            roi=face_image[y:y+h,x:x+w]
            face_image = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
            face_cords.append([x,y,w,h])
            cv2.imwrite("extracted" + '.png', roi)
        if len(face_cords) >= 1:
            face_detection = len(face_cords)
            return {"image": face_image, "detection": face_detection, "roi": roi}
            
        else:
            return {"image": face_image, "detection": face_detection}
    except Exception as e:
        return{"image": image, "detection": 0}

# Function to detect eyes
def detect_eyes(image):
    try:
        eye_image = image.copy()
        eye_rect = eye_cascade.detectMultiScale(eye_image, scaleFactor=1.2, minNeighbors=5)
        eye_cords = []
        eye_detection = 0
        for (x, y, w, h) in eye_rect:
            if len(eye_rect) == 2:
                cv2.rectangle(eye_image, (x, y), (x + w, y + h), (255, 255, 255), 10)
                eye_cords.append([x,y,w,h])
        if len(eye_cords) >= 1:
            eye_detection = len(eye_cords)
        return {"image": eye_image, "detection": eye_detection}
    except Exception as e:
        return{"image": image, "detection": 0}

def detect_glass(image):
    try:
        glass_image = image.copy()
        glass_rect = glass_cascade.detectMultiScale(glass_image, scaleFactor=1.2)
        # print("Glass Detection = " + str(len(glass_rect)))
        return {"image": glass_image, "detection": len(glass_rect)}
    except Exception as e:
        return{"image": image, "detection": 0}

def detect_profile(image):
    try:
        profile_image = image.copy()
        profile_rect = side_profile.detectMultiScale(profile_image, scaleFactor=1.2)
        # print("Profile Detection = " + str(len(profile_rect)))
        return {"image": glass_image, "detection": len(glass_rect)}
    except Exception as e:
        return{"image": image, "detection": 0}

## *CNN Network*

In [7]:
def cnn_model():
    try:
        json_file = open("models/model.json", "r")
        loaded_model = json_file.read()
        json_file.close()

        load_model = model_from_json(loaded_model)
        load_model.load_weights("models/model.h5")

        test_image = img.load_img('extracted.png', target_size=(200, 200, 3))
        test_image = img.img_to_array(test_image)
        test_image = np.expand_dims(test_image, axis=0)
        result = load_model.predict(test_image)
        if result[0][0] == 1:
            return {"message": "Real Face"}
        else:
            return {"message":"Fake Face"}
    except Exception as e:
        return {"message":e}


## *SSD Detection*

In [8]:
def ssd(image):
    try:
        id = 0
        className = ["person"]
        classFile = "config/coco.names"
        configPath = "config/ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt"
        weightsPath = "config/frozen_inference_graph.pb"
        net = cv2.dnn_DetectionModel(weightsPath, configPath)
        net.setInputSize(200, 200)
        net.setInputScale(1.0 / 127.5)
        net.setInputMean((127.5, 127.5, 127.5))
        net.setInputSwapRB(True)
        classIDs, confs, bbox = net.detect(image)
        id = len(classIDs)
        return {"detection": id}
    except Exception as e:
        return {"detection": e}

In [9]:
def classification(image):
    try:
        original_height, original_width = image.shape[:2]
        image_resize = cv2.resize(image, (200, 200), interpolation=cv2.INTER_AREA)
        brightness_corrected = brightness_correction(image_resize)
        filtered_image = gamma_correction(brightness_corrected)
        filtered_image = cv2.cvtColor(filtered_image, cv2.COLOR_BGR2GRAY)
        normalized_image = normalize_image(filtered_image)
        image_resize = cv2.resize(normalized_image, (original_width, original_height), interpolation=cv2.INTER_LANCZOS4)
        glass_detction = detect_glass(image_resize)
        profile_detection = detect_profile(image_resize)
        face = detect_face(image_resize)
        original_eye = detect_eyes(image)
        filtered_eye = detect_eyes(image_resize)
        # print("Face " + str(face["detection"]))
        # print("Filtered eye " + str(filtered_eye["detection"]))
        # print("original_eye " + str(original_eye["detection"]))
        if face["detection"] >= 1 and filtered_eye["detection"] == 2:
            return {"message": "Real Face"}
        elif face["detection"] == 1 and filtered_eye["detection"] >=0 and original_eye["detection"] > 0:
            return {"message": cnn_model()["message"]}
        elif filtered_eye["detection"] > 0 and original_eye["detection"] == 2:
            return {"message": cnn_model()["message"]}
        elif filtered_eye["detection"] == 2:
            return {"message": cnn_model()["message"]}
        elif profile_detection["detection"] > 0 or (glass_detction["detection"] > 0 and glass_detction["detection"] < 3 and face["detection"] >= 1):
            return {"message": cnn_model()["message"]}
        else:
            return {"message":"Fake Face"}
    except Exception as e:
        return {"message":e}

## *Classify real vs Fake image*

In [10]:
# path = "input/glasses"
# i = 1
# for image_path in os.listdir(path):
#     input_path = os.path.join(path, image_path)
#     input_image = cv2.imread(input_path)
#     detection = ssd(input_image)["detection"]
#     # print("Detection = " + str(detection))
#     if detection < 3:
#         # print(str(i) +" : "+ str(image_path))     
#         message = classification(input_image)["message"]
#         print(str(i) + "::File - " + str(image_path) + ", Message - " + str(message))
#     else:   
#         print(str(i) + "::File - " + str(image_path) + ", Message - Fake Face")
#     i += 1   

In [11]:
# path = "input/Fake_Faces"
# i = 1
# for image_path in os.listdir(path):
#     input_path = os.path.join(path, image_path)
#     input_image = cv2.imread(input_path)
#     detection = ssd(input_image)["detection"]
#     # print("Detection = " + str(detection))
#     if detection < 3:
#         # print(str(i) +" : "+ str(image_path))     
#         message = classification(input_image)["message"]
#         print(str(i) + "::File - " + str(image_path) + ", Message - " + str(message))
#     else:
#         print(str(i) + "::File - " + str(image_path) + ", Message - Fake Face")
#     i += 1   

In [12]:
# path = "input/Real_Faces"
# i = 1
# for image_path in os.listdir(path):
#     input_path = os.path.join(path, image_path)
#     input_image = cv2.imread(input_path)
#     detection = ssd(input_image)["detection"]
#     if detection < 3:
#         message = classification(input_image)["message"]     
#         print(str(i) + "::File - " + str(image_path) + ", Message - " + str(message))
#     else:
#         print(str(i) + "::File - " + str(image_path) + ", Message - Fake Face")
#     i += 1

In [16]:
input_image = cv2.imread("input/2020-12-11-203654.jpg")
detection = ssd(input_image)["detection"]
if detection < 3:
    message = classification(input_image)["message"]
    print("Message - " + str(message))
else:
    print("Message - Fake Face")

Message - Real Face
